# Mixed-integer programming

Operations research often involves models where you need discrete decisions. 

Suppose Iain wants to carry items to the pawn shop to get some extra cash. He has $N$ items, each with a weight $w_i$ and a price $p_i$. Iain hasn't been to the gym lately, so he can only carry $C$ kilos. How does he choose what to bring with him?

We can model this as an integer optimization problem:

\begin{align*}
\max& \sum_{i=1}^N p_i x_i \\
\text{s.t.}& \sum_{i=1}^N w_i x_i \leq C \\
& x_i \in \{0,1\} \quad \forall i = 1,\ldots,N
\end{align*}


## Example

In particular, consider the following (small) knapsack problem:

\begin{align*}
    \max\:& x + y \\
    \text{s.t.}\:& x + 2y \leq 1.5 \\
    & x, y \in \{0,1\}
\end{align*}

How would you solve this? 

## Branch and Bound Tree

The simple way is just to consider each possible value for $x$ and $y$ and compare the cost.

![alt text](img/tree_1.png)

In the general case, this would lead to $2^N$ possible collections of items. After Iain has weighed all of them (and verified that he can lift them at once), he just chooses the best set.


Let's visualize this approach as a search tree:

![alt text](img/tree_2.png)

It's rooted at what we call the _relaxation_: none of variables have integrality enforced. As we go down leaves of the tree, we add pick a variable to _branch_ on, and create two descended nodes that fix that variable to one of its possible values. If we follow the tree all the way to the bottom, we reach our enumeration from before.

As we go down the arcs of the tree we restrict our problem more and more, we must have that:

>If node ``q`` is descended from node ``p``, we must have that the optimal cost of subproblem ``q`` is no more than that for node ``p``

This leads us to a powerful tool in solving these enumeration problems: 

>If I can show you that the optimal cost for subproblem ``q`` is _less_ than the optimal cost for the original problem, the same is true for any descendent of ``q``. 


That is, we can _prune_ the tree and safely discard some nodes, kind of like this:

![alt text](img/tree_3.png)

## Worked Example

\begin{align*}
    \max\quad& x + y \\
    \text{s.t.}\quad& x + 2y \leq 1.5 \\
    & 0 \leq x, y \leq 1 \\
    & x, y \in \{0,1\}
\end{align*}

What does the relaxation (no integrality restrictions) of this problem look like?

![fig1](http://www.mit.edu/~huchette/.tmp/fig1.svg)

First, we solve the LP relaxation and get $(x^*,y^*) = (1,0.25)$. 

![fig1](http://www.mit.edu/~huchette/.tmp/fig2.svg)

This isn't integer feasible, so we branch on $y$. The subproblem with $y = 1$ is infeasible, and the subproblem with $y = 0$ is feasible with solution $(x^*,y^*) = (1,0)$. This is integer feasible, so we update our lower bound. We've also exhausted the tree, so we have our optimal solution!

![fig1](http://www.mit.edu/~huchette/.tmp/fig3.svg)

The branch-and-bound scheme can end up solving many subproblems, so for it to work well, we need to _prune_ large portions of the tree. 

## Branch and bound
We'll keep track of a global _lower bound_ $LB$ for our problem. Each node ``q`` will have an upper bound $UB_q$ that it inherents from its parent. If we get to the point where we have solved all subproblems (or, ideally, pruned off a great deal of them), we know that we're optimal. To do this we'll also keep track of a list $L$ of subproblems left to solve; initially, it's just the relaxation. The procedure is:

While $L$ is not empty, pick a subproblem ``q`` out of our list $L$ and solve it. 
1. ``if`` ``q`` is infeasible, ``continue``
2. ``if`` the solution is integer feasible, update the lower bound $LB$ if the cost is higher than what we had before
3. ``if``  the relaxation value is less than our global $LB$ ``continue``
4. ``else`` pick a non-integer variable $i$ and _branch_ by adding two subproblems to $L$: 
    * One with $x_i = 0$
    * Another with $x_i = 1$

Branch-and-bound is sometimes called an _implicit enumeration_ scheme because of step 3: we avoid solving any subproblems that we can prove won't produce the optimal solution.

** The "magic" of modern MIP solvers largely comes down to pruning massive portions of the tree **


## Stuff you should care about

1. Cuts (improve the _upper bounds_)
2. Heuristics (improve the _lower bounds_)
3. Presolve (_simplify_ everything before we build the tree)
4. Branching strategies (construct the tree in a smart way)

## 1. Cuts

Cuts are inequalities that are _valid_ (don't cut off any feasible integer points) and _strengthen_ the formulation (chop off some of the region feasible for the relaxation)

Three main types of cuts:

1. General purpose (e.g. CG, split, MIR)
2. Structure-specific (e.g. knapsack cover, clique)
3. Problem-specific (whatever _you_ cook up)

### Example 

Let's go back to our knapsack example and imagine that had been a bit smarter and realized that $x + \frac{4}{3}y \leq 1$ is feasible for all integer feasible points. If we add this to our formulation, we have the optimal solution at the root node!

![fig1](http://www.mit.edu/~huchette/.tmp/fig4.svg)

## 2. Heuristics

You can add integer feasible solutions into the branch-and-bound scheme to improve the lower bound. These can come from

1. Problem-specific heuristics
2. Neighborhood search
3. Rounding or "polishing"

### Example

After we've already solved it twice, it's easy to see that the optimal solution for our knapsack is $(1,0)$. So, we can just start out the branch-and-bound procedure with $LB = 1$.

More generally, if we're solving a knapsack problem, one simple heuristic is to add a _greedy solution_ where you iteratively add the best available item to the sack until you run out of room. This will often be a very good solution, and is a simple example of a problem-specific heuristic scheme.

![fig5](http://www.mit.edu/~huchette/.tmp/fig5.svg)

## 3. Presolve

Modern MIP solvers have a litany of techniques to simplify problems _before_ constructing the tree. 

* Variable/constraint bounds tightening
* Logical inferences
* General cleanup (remove redundant variables, constraints)

By shrinking problems, presolve potentially shrinks

* the B&B tree (fewer binary variables)
* The node solve time (smaller LP relaxations)

### Example

For our problem
\begin{align*}
    \max\:& x + y \\
    \text{s.t.}\:& x + 2y \leq 1.5 \\
    & x, y \in \{0,1\}
\end{align*}

we can perform bounds tightening to get

\begin{align*}
    \max\:& x + y \\
    \text{s.t.}\:& x + 2y \leq 1 \\
    & x, y \in \{0,1\}
\end{align*}

![fig1](http://www.mit.edu/~huchette/.tmp/fig6.svg)

Then, we can reason that, since $x,y \in \{0,1\}$, we cannot have $y=1$, and so we can reduce our problem to 

\begin{align*}
    \max\:& x \\
    & x \in \{0,1\}
\end{align*}

Which we can then solve in closed form.

## 4. Branching strategies

Different paths through the B&B tree can see drastically different performance:

* Missing a node that produces a good feasible solution will mean you add nodes to your list you otherwise would have pruned
* Choosing nodes that would otherwise be pruned just slows you down

How do we know _a priori_ what a good branching strategy is, though? 

We don't really, but you can make informed guesses. 

Consider another small problem:
\begin{align*}
    \max\:& y \\
    \text{s.t.}\:& -x + y \leq \frac{1}{3} \\
    & x + y \leq \frac{4}{3} \\
    & 0 \leq x,y \leq 1 \\
    & x, y \in \{0,1\}
\end{align*}
If we solve the relaxation, we get an optimal solution of $(x^*,y^*) = (\frac{1}{2},\frac{5}{6})$.

![fig1](http://www.mit.edu/~huchette/.tmp/fig7.svg)

Now we have a choice: both coordinates are fractional, so we can branch on either. If we branch on $y$, we get

![fig1](http://www.mit.edu/~huchette/.tmp/fig8.svg)

The branch with $y=0$ gives us an optimal solution in $(0,1)$, and the $y=1$ branch is infeasible, so we have no more branches to take and we are done. But if we had branched on $x$ instead,

![fig1](http://www.mit.edu/~huchette/.tmp/fig9.svg)

The optimal solution for both branches is still fractional, so we have to branch again. If we have, say, 1000 binary variables instead of just two, we see how branching strategies can have an enormous effect on performance.

# Implementation in JuMP

Let's solve our simple knapsack problem and see what the solver spits out.

In [ ]:
using JuMP, Gurobi
m = Model(solver=GurobiSolver())

@variable(m, x, Bin) # or Int
@variable(m, y, Bin)

@constraint(m, x + 2y ≤ 1.5)
@objective(m, Max, x + y)

solve(m)

This is kind of dull since Gurobi solves this before we ever get to the branch-and-bound tree! Let's cook up a problem that's a little more interesting. What about more items, and more knapsacks! If $N=100$, naïve enumeration would create $2^{100}$ nodes, which would take quite some time. How does the solver actually tackle it?

In [ ]:
srand(100)
N = 100

m = Model(solver=GurobiSolver())
@variable(m, x[1:N], Bin)
for _ in 1:10
    @constraint(m, dot(rand(N), x) ≤ N / 50)
end

@objective(m, Max, dot(rand(N), x))

solve(m)

## Understanding solver output

First, it solves the LP relaxation and reports back:
```
Root relaxation: objective 4.014179e+00, 18 iterations, 0.00 seconds
```
Now it explores the branch-and-bound tree, and updates us as it goes along. Let's look at just the first line:
```
    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.01418    0    7    2.35937    4.01418  70.1%     -    0s
```
We see that the information is broken down into four main columns:

1. ``Nodes``: Global node information
    * how many nodes have we looked at
    * how many do we have in our queue
2. ``Current Node``
    * objective
    * depth in the tree
    * number of noninteger variables in the solution
3. ``Objective Bounds``
    * Best incumbent (lower bound)
    * node upper bound
    * the gap between the two
4. ``Work``
    * average simplex iterations per node
    * total elapsed time

Finally, we get a neat summary of the cutting planes Gurobi found useful:
```
Cutting planes:
  Gomory: 3
  Cover: 2
  MIR: 5
```
All told, we explored 190  nodes, much less than the $2^{100}$ we were worried about. All this only took 698 simplex iterations and 0.21 seconds.

Now what about those ``H``s that appear? That tells us that Gurobi ran a heuristic and found a new best solution. You can see for yourself, as the incumbent value increases while the bound remains the same:
```
    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.01418    0    7    2.35937    4.01418  70.1%     -    0s
H    0     0                       3.3780464    4.01418  18.8%     -    0s
```
You'll also sometimes see a ``*`` instead of the ``H``, which says that the feasible solution came from branching instead of heuristics.

# Solver parameters: Should you bother?

Gurobi (and other high-quality solvers) allow you to tweak a wide range of different parameters; _sometimes_ tuning these can drastically improve performance. It can be kind of intimidating, though: Gurobi has over 100 parameters, so which are the important ones?

Some useful ones:

* ``TimeLimit``: How long the solver will run before giving up.
* ``NodeLimit``: How many nodes to explore before giving up.
* ``MIPGap``: Termination criterion for relative gap $\frac{UB-LB}{LB}$.
* ``MIPFocus``: High-level controls on solver priority (proving optimality or increasing bound or finding optimal solution).
* ``Heuristics``: Determines the amount of time spent in MIP heuristics.
* ``Cuts``: Controls the aggressiveness of cut generation.
* ``Presolve``: Controls the presolve level (conservative or aggressive).

Is that it? Well, no, but you probably need domain knowledge about your problem to go much further. There's an alternative: Gurobi has a parameter tuning feature you can try to "learn" good parameter settings for a particular model. Try it out if you aren't quite happy with your performance.

## Sudoku

![Sudoku](http://upload.wikimedia.org/wikipedia/commons/f/ff/Sudoku-by-L2G-20050714.svg)

**Sudoku** is a number puzzle played on a 9x9 grid. The challenge is to place a digit between 1 and 9 inclusive in each empty cell, such that the completed grid obeys the following rules:

* Each row contains the numbers 1 to 9 once and only once.
* Each column contains the numbers 1 to 9 once and only once.
* Each 3x3 subgrid contains the numbers 1 to 9 once and only once.

In [ ]:
init_vals = [
5 3 0 0 7 0 0 0 0
6 0 0 1 9 5 0 0 0
0 9 8 0 0 0 0 6 0
8 0 0 0 6 0 0 0 3
4 0 0 8 0 3 0 0 1
7 0 0 0 2 0 0 0 6
0 6 0 0 0 0 2 8 0
0 0 0 4 1 9 0 0 5
0 0 0 0 8 0 0 7 9
]

using PyPlot
# Displays a sudoku matrix. If an initial state is provided, 
# then the new numbers are displayed in red. 
function display_sudoku(sudoku, initial_state=zeros(9,9))
    fig, ax = subplots()
    ax[:axis]("off")
    # Make sudoku basic 9x9 grid.
    for i in 0:9
        plot([i,i], [0, 9], color = "black")
        plot([0, 9], [i,i], color = "black")
    end
    # Make thicker lines to separate the 3x3 subgrids.
    for i in 0:3:9
        plot([i,i], [0,9], color = "black", linewidth = 3)
        plot([0,9], [i,i], color = "black", linewidth = 3)
    end
    # Display the values at the right square.
    for i in 0:8
        for j in 0:8 
            value = Int(sudoku[9-j,i+1])
            old_value = initial_state[9-j,i+1]
            # If an initial state is not provided (set to zero) or the value of the 
            # square did not change, the color is set to black. Otherwise, it becomes red.
            col = (sum(initial_state) == 0 || old_value == value) ? "black" : "red"
            if value > 0 # Omit zero values  
                text(i + 0.35, j + 0.33, value, size = 16, color = col)
            end
        end
    end
end
display_sudoku(init_vals)

The most natural formulation of this problem would probably be something like

$$x_{i,j} \in \{1, 2, \dots, 9\}$$

which is of course something we can do with integer programming:

$$1 \leq x_{i,j} \leq 9, ~ integer$$

The challenge now is the constraints. One observation is that the numbers 1 to 9 add up to 45, so we could try something like:

$$ \sum_{j=1}^9 x_{i,j} = 45 $$

In [ ]:
using JuMP, Gurobi
function sudoku1()
    sudoku = Model(solver=GurobiSolver(OutputFlag=0))
    @variable(sudoku, 1 ≤ x[1:9,1:9] ≤ 9, Int)
    
    @constraints(sudoku, begin
        rows[i=1:9], sum(x[i,:]) == 45
        cols[j=1:9], sum(x[:,j]) == 45
        subgrid[i=1:3:7,j=1:3:7], sum(x[i:i+2,j:j+2]) == 45
    end)
    
    for i in 1:9, j in 1:9
        if init_vals[i,j] != 0
            @constraint(sudoku, x[i,j] == init_vals[i,j])
        end
    end
    
    solve(sudoku)
    
    getvalue(x)
end
@time soln1 = sudoku1();

In [ ]:
display_sudoku(soln1, init_vals)

Well that didn't work. We might be able to make this work, but we'd need many more constraints. Instead, let's change our **variables**: $x_{i,j,k} = 1$ iff the number $k$ will appear in cell $(i,j)$. We can now use our 0-1 integer programming knowledge to model the problem. Consider the "row" constraints: we want each number to appear once and only once. This is equivalent to saying that

$$\sum_{j=1}^9 x_{i,j,k} = 1 \quad \forall i, k$$

We can now model this as a $9\times 9\times 9$ dimensional problem - thats a lot of binary variables, surely Gurobi won't like that!

In [ ]:
function sudoku2()
    sudoku = Model(solver=GurobiSolver())

    @variable(sudoku, x[i=1:9, j=1:9, k=1:9], Bin)

    @constraints(sudoku, begin
        # Constraint 1 - Exactly one value appears in each cell.
        cell[i=1:9, j=1:9], sum(x[i,j,:]) == 1
        # Constraint 2 - Each value appears in each row exactly once.
        row[i=1:9, k=1:9], sum(x[i,:,k]) == 1
        # Constraint 3 - Each value appears in each column exactly once.
        col[j=1:9, k=1:9], sum(x[:,j,k]) == 1
        # Constraint 4 - Each value appears in each 3x3 subgrid exactly once.
        subgrid[i=1:3:7,j=1:3:7,val=1:9], sum(x[i:i+2,j:j+2,val]) == 1
    end)

    # Fix given values. 
    for row in 1:9, col in 1:9
        if init_vals[row,col] != 0
            @constraint(sudoku, x[row, col, init_vals[row, col]] == 1)
        end
    end
    
    solve(sudoku)
    
    getvalue(x)
end
@time soln2 = sudoku2();

In [ ]:
# soln2 is a 9x9x9 array with values 0 or 1. 
# First we need to transform it to a 9x9 matrix with the right values 1,...,9.
soln2_array = sum(i * soln2[:,:,i] for i in 1:9)
display_sudoku(soln2_array, init_vals)

## Presolving the Problem
Can you see the lines
```
Optimize a model with 358 rows, 729 columns and 2950 nonzeros
Presolve removed 358 rows and 729 columns
```
? Gurobi was able to use logic to deduce the value of every variable - no linear relaxation required! This "magic" is actually how a human might solve it. Consider the following:

We know that $x_{2,6,5}$ is fixed at 1 because it is one of the provided values. So we can actually replace $x_{2,6,5}$ wherever it appears in the constraints with the constant 1:

"The value 5 must appear in row 2":
$$x_{2,1,5} + x_{2,2,5} + x_{2,3,5} + x_{2,4,5} + x_{2,5,5} + x_{2,6,5} + x_{2,7,5} + x_{2,8,5} + x_{2,9,5} = 1$$
$$\rightarrow x_{2,1,5} + x_{2,2,5} + x_{2,3,5} + x_{2,4,5} + x_{2,5,5} + 1 + x_{2,7,5} + x_{2,8,5} + x_{2,9,5} = 1$$
$$\rightarrow x_{2,1,5} + x_{2,2,5} + x_{2,3,5} + x_{2,4,5} + x_{2,5,5} + x_{2,7,5} + x_{2,8,5} + x_{2,9,5} = 0$$

"The value 5 must appear in column 6":
$$x_{1,6,5} + x_{2,6,5} + x_{3,6,5} + x_{4,6,5} + x_{5,6,5} + x_{6,6,5} + x_{7,6,5} + x_{8,6,5} + x_{9,6,5} = 1$$
$$x_{1,6,5} + 1 + x_{3,6,5} + x_{4,6,5} + x_{5,6,5} + x_{6,6,5} + x_{7,6,5} + x_{8,6,5} + x_{9,6,5} = 1$$
$$x_{1,6,5} + x_{3,6,5} + x_{4,6,5} + x_{5,6,5} + x_{6,6,5} + x_{7,6,5} + x_{8,6,5} + x_{9,6,5} = 0$$

and so on. Because all those other variables can only be 0 or 1, and their sum is 0, they must all be 0. Thus Gurobi presolve can perform the following procedure:
1. Replace all the fixed values with constants
2. Use constraints to fix variables, e.g. at 0 (or 1)
3. Go to 1 unless step 2 did nothing.

A small problem arises when there are multiple solutions to the problem - a random selection has to be made. Gurobi will treat this case as "trying to find a feasible solution" - it will fix a variable, and follow the implications through.

Gurobi can do similar presolving implications for continuous decisions too, e.g.
$$x \in \{ 0, 1 \}$$
$$x \leq \frac{1}{2}$$

will presolve to $x = 0$: Gurobi knows that all the variables are integer, so we can safe round down the right-hand-side to the closest integer.

>**\[Exercise\]**: Pre-solve

> What if the right-hand-side is more complicated? What can you do to "tighten" the following constraint and the variables in it?

> $$ 6x_1 + 10x_2 + 12x_3 + 19x_4 \leq 15$$

> Hints:
> * Can you find common factors in the coefficients?
> * Can you round anything?
> * Can you prove things about the variable values?

> (all variables are binary)

>**\[Solution\]**: Pre-solve

> * $6x_1 + 10x_2 + 12x_3 + 19x_4 \leq 15$
> * $x_4$ must be 0, because if $x_4$ was 1 then the constaint would be violated
> * $6x_1 + 10x_2 + 12x_3 \leq 15$
> * All coefficients on left are multiples of 2, so divide through by 2
> * $3x_1 + 5x_2 + 6x_3 \leq 7.5$
> * All coefficients are integer, all variables are binary, so can round down the RHS.
> * $3x_1 + 5x_2 + 6x_3 \leq 7$
> * We note though that only one of these variables can be one at a time, or the constraint would be violated, so the best we can do is actually
> * $x_1 + x_2 + x_3 \leq 1$

## Big $M$s and logical implications

We often need to model the following type of constraint:

> The constraint $a^Tx \leq b$ holds if and only if $z=1$

An example of this appears in network problems:

> We have a network with a 'source' and a 'sink', and want to maximize flow between them across the network. We have a set of arcs we can build, at a cost, so our objective is to maximize profit from the flow less the cost of building these arcs.

If we say that $x_{i,j}$ is the flow from $i$ to $j$, and $z_{i,j}$ is a binary decision for whether we build the arc or not, we will have the constraint

$$ x_{i,j} > 0 \Longrightarrow z_{i,j}=1 $$

One way to express this as a linear equality is to write

$$ 0 \leq x_{i,j} \leq M z_{i,j} $$

where $M$ is a **sufficiently large constant**. That means that $M$ is greater than the largest value $x_{i,j}$ would take in an optimal solution.

In some problems, picking a value for $M$ is easy - for example, if arcs have a maximum capacity $C$, set $M \leftarrow C$. In other cases, it is not clear *a priori* what the largest value $x_{i,j}$ can take is. It may be tempting to just choose a large value, like $M\leftarrow 10^{10}$, just to be safe. However this can make your MILP much harder to solve, as we will now demonstrate.

### Bad Big Ms

Consider the following toy problem:

$$
\begin{align*}
\max\quad& x_1 + x_2 + x_3 \\
\text{s.t.}\quad& Ax \leq b \\
& 0 \leq x_i \leq M_i z_i \\
& \sum_{i=1}^3 z_i \leq 1 \\
& (x,z) \in \mathbb{R}^3 \times \{0,1\}^3
\end{align*}
$$

or, in words, pick the single largest $x_i$, subject to the linear constraints $Ax\leq b$. These linear constraints will imply some bounds on $x_i$, although it may be hard to find them. If we did know them, we could set $M_i$ to those bounds. Since we don't, we'll need to pick a "sufficiently large" $M$ from our understanding of the problem. Lets see what the solution of the linear relaxation of this problem looks like as we vary $M$.

In [ ]:
using Gadfly, Interact, Compose
function example1(M)
    mod = Model(solver=GurobiSolver(OutputFlag=0))
    @variable(mod, x[1:3] ≥ 0)
    @variable(mod, z[1:3], Bin)
    @objective(mod, Max, sum(x))
    
    # We will make our Ax≤b a very simple set!
    @constraint(mod, x[1] ≤ 4)
    @constraint(mod, x[2] ≤ 8)
    @constraint(mod, x[3] ≤ 9)
    
    @constraint(mod, bigm[i=1:3], x[i] ≤ M*z[i])
    @constraint(mod, sum(z) ≤ 1)
    
    solve(mod, relaxation=true)
    relax = getobjectivevalue(mod)
    
    solve(mod)
    opt = getobjectivevalue(mod)
    
    # MILP objective is 9, compare with LP objective
    gap = (relax-opt) / opt

    return 100gap, getvalue(x)[:], getvalue(z)[:]
end

Mrange = 0:3:30
Gadfly.plot(
    x=Mrange,y=[example1(M)[1] for M in Mrange],
    Geom.line,Geom.point,
    xintercept=[9], Geom.vline(color="orange"),
    yintercept=[0], Geom.hline(color="black"),
    Guide.xlabel("M"),
    Guide.ylabel("Gap (%)"),
)